## Speedup when running PyTorch models on ONNX

In this notebook, we show the speed-up on inference time that we achieve when migrating a Resnet18 from PyTorch to ONNX.

In [3]:
!pip install onnx -q
!pip install onnxruntime -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.3 MB/s eta 0:00:00


In [4]:
import torch.onnx
import torchvision.models as models

# Initialize model
model = models.resnet18(pretrained=True)

# Set model to evaluation mode
model.eval()

# Define dummy input
x = torch.randn(1, 3, 224, 224)

# Export to ONNX
torch.onnx.export(model, x, "resnet18.onnx")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 58.0MB/s]



### Inference time for PyTorch model



In [8]:
%%time
output = model(x)

CPU times: user 88.4 ms, sys: 516 µs, total: 88.9 ms
Wall time: 92 ms


In [ ]:
import onnxruntime as ort
import numpy as np
import torch

# Initialize ONNX Runtime session
ort_session = ort.InferenceSession("resnet18.onnx")

# Prepare input data (same shape as the dummy input used for exporting the model)
input_name = ort_session.get_inputs()[0].name
input_shape = (1, 3, 224, 224)
input_data = np.random.randn(*input_shape).astype(np.float32)

### Inference time for ONNX model

In [11]:
%%time
# Run inference
ort_inputs = {input_name: input_data}
ort_outs = ort_session.run(None, ort_inputs)

CPU times: user 59.8 ms, sys: 0 ns, total: 59.8 ms
Wall time: 63.9 ms


In [12]:
# Extract output
output_data = ort_outs[0]

# Convert ONNX Runtime output to PyTorch tensor if needed
output_tensor = torch.from_numpy(output_data)
output_tensor.shape

torch.Size([1, 1000])